In [31]:
import numpy as np
import pandas as pd
import bokeh.plotting as bp
from bokeh.layouts import gridplot

En primera base se probó con la señal del par EUR-USD para ver esta como se comportaba, ahora probaremos con varias señales de entrada, para ver como se comporta la señal EUR-USD.

Para empezar se probará con los pares EUR/USD y GBP/USD para predecir la señal EUR/USD

In [32]:
#Se lee la señal correspondiente a GBP vs USD para el mes de enero de 2018
data_GBPvsUSD = pd.read_csv("../data/TrueFX/GBP-USD/GBPUSD-preprocesado-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

#Se lee la señal correspondiente a EUR vs USD para el mes de enero de 2018
data_EURvsUSD = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_EURUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

In [33]:
data_GBPvsUSD[:10]

,bid,ask
date,,
2018-01-02 00:00:40,1.34979,1.35176
2018-01-02 00:01:00,1.34918,1.35208
2018-01-02 00:01:10,1.34915,1.35354
2018-01-02 00:01:20,1.34915,1.35386
2018-01-02 00:01:30,1.34938,1.35440
2018-01-02 00:01:40,1.34938,1.35386
2018-01-02 00:01:50,1.34942,1.35300
2018-01-02 00:02:10,1.34939,1.35300
2018-01-02 00:02:30,1.34979,1.35154


In [34]:
data_EURvsUSD[:10]

,bid,ask
date,,
2018-01-01 23:58:00,1.20093,1.20115
2018-01-01 23:58:10,1.20093,1.20111
2018-01-01 23:58:20,1.20082,1.20111
2018-01-01 23:58:30,1.20043,1.20110
2018-01-01 23:58:50,1.20011,1.20113
2018-01-01 23:59:00,1.20015,1.20110
2018-01-01 23:59:10,1.20028,1.20110
2018-01-01 23:59:50,1.20015,1.20110
2018-01-02 00:00:40,1.20015,1.20105


In [35]:
print('tamaño de la señal EUR/USD', data_EURvsUSD.shape)
print('tamaño de la señal GBP/USD', data_GBPvsUSD.shape)

tamaño de la señal EUR/USD (183535, 2)
tamaño de la señal GBP/USD (185443, 2)


como se puede ver, las dos señales no tienen el mismo tamaño, como la señal predominante es el par EUR/USD, el indice de este será el tenido en cuenta para la construcción de la multiseñal.

procedemos a cargar la clase "EUtilities" hecha con anterioridad para tratar estos datos...

In [36]:
%run ../code/EUtilities.py
EU = EUtilities()

para poder utilizar la función "multi_signal" se deben cumplir ciertas caracteristicas, entre ellas, que los dataframe a utilizar tenga su columna llamada "bid"...

In [37]:
#se construye el dataset EUR/USD con las dos señales en el, para su posterior uso
data_EURvsUSD = EU.multi_signal(data_EURvsUSD,data_GBPvsUSD)

In [38]:
data_EURvsUSD[:10]

,bid,ask,bidnew
date,,,
2018-01-01 23:58:00,1.20093,1.20115,1.20093
2018-01-01 23:58:10,1.20093,1.20111,1.20093
2018-01-01 23:58:20,1.20082,1.20111,1.20082
2018-01-01 23:58:30,1.20043,1.20110,1.20043
2018-01-01 23:58:50,1.20011,1.20113,1.20011
2018-01-01 23:59:00,1.20015,1.20110,1.20015
2018-01-01 23:59:10,1.20028,1.20110,1.20028
2018-01-01 23:59:50,1.20015,1.20110,1.20015
2018-01-02 00:00:40,1.20015,1.20105,1.34979


ya con la multiseñal se procede a crear el X...

In [39]:
def build_dataset(df, window, bid_col, binary_target=False, delete_constant_values=True, PNL=False):
    """
    función para construir un data set
    window: tamaño de la ventana a utilizar para construir el dataset
    df: dataframe, con columna bid y ask.
    binary_target: si desea clasificar, este arroja 2 si el valor se mantiene,
    1 si el valor sube y 0 si este baja.
    delete_constant_values: default: True, elimina los valores que se mantienen
    
    retorna:
    X: dataset, con columna de PNL si así se especifica(default: False)
    y: target
    bt: binary target, default: False
    """

    import pandas as pd
    import numpy as np
    result = []
    binary = [] #para la columna objetivo binaria
    pnl_buy = [] #almacenar el pnl en caso de compra
    pnl_sell = [] #almacenar el pnl en caso de venta
    signal = df[bid_col]
    ask = df.ask
    indx = signal.index[window-1:-1] #se toman los indicen que quedarán al final
    for i in range(len(signal)-window):
        
        if delete_constant_values == True:
            if signal[i+window] != signal[i+window-1]:

                result.append(signal[i: i + window+1])
                if PNL == True: 
                    pnl_sell.append(signal[i+window-1] - ask[i+window]) #calcular pnl en caso de venta-compra
                    pnl_buy.append(signal[i+window] - ask[i+window-1]) #calcular pnl en caso de compra-venta
                
                if binary_target == True:
                    if signal[i+window] < signal[i+window-1]: binary.append(0) # 0 si baja
                    if signal[i+window] > signal[i+window-1]: binary.append(1) # 1 si sube
                        
            else: indx = indx.delete(len(result))
                
        else:

            result.append(signal[i: i + window+1])
            if PNL == True:
                pnl_sell.append(signal[i+window-1] - ask[i+window]) #calcular pnl en caso de venta-compra
                pnl_buy.append(signal[i+window] - ask[i+window-1]) #calcular pnl en caso de compra-venta
            
        if binary_target == True and delete_constant_values == False:
            if signal[i+window] == signal[i+window-1]: binary.append(2) # 2 si se mantiene
            if signal[i+window] < signal[i+window-1]: binary.append(0) # 1 si baja
            if signal[i+window] > signal[i+window-1]: binary.append(1) # 0 si sube
    
    data = pd.DataFrame(np.array(result), index=indx)
    y = np.array(data.iloc[:,window])
    data = data.drop(window,axis=1)
    if PNL == True:
        data['PNL_0'] = pnl_sell
        data['PNL_1'] = pnl_buy 
    if binary_target == True: return data, y, np.array(binary)
    else: return data, y

In [40]:
#los datos del par EUR/USD 
X_EURUSD, y_EURUSD, bt_EURUSD = build_dataset(data_EURvsUSD,bid_col='bid',window=3,binary_target=True,PNL=True
                                             ,delete_constant_values=True)
#los datos del par GBP/USD para luego unirlos y formar los datos X para entrenar
X_GBPUSD,_ = build_dataset(data_EURvsUSD,bid_col='bidnew',window=3,binary_target=False
                              ,delete_constant_values=True)

In [41]:
X_EURUSD.shape, X_GBPUSD.shape

((156579, 5), (161099, 3))

debido a que son diferentes señales, al habilitar la opción de <i>"delete_constant_values"</i> en la función <i>"build_dataset"</i> esto hace que las señales resultantes no sean iguales en tamaño, por ende para entrenar se cogeran los valores de los indices de la señal a medir, si este no está en la otra señal, se coloca lo de la señal EUR/USD

In [42]:
X_GBPUSD = X_GBPUSD.loc[X_EURUSD.index]
X_EURUSD.shape, X_GBPUSD.shape

((156579, 5), (156579, 3))

In [43]:
X_GBPUSD[:10]

,0,1,2
date,,,
2018-01-01 23:58:20,1.20093,1.20093,1.20082
2018-01-01 23:58:30,1.20093,1.20082,1.20043
2018-01-01 23:58:50,1.20082,1.20043,1.20011
2018-01-01 23:59:00,1.20043,1.20011,1.20015
2018-01-01 23:59:10,1.20011,1.20015,1.20028
2018-01-02 00:00:40,1.20028,1.20015,1.34979
2018-01-02 00:01:20,1.20030,1.34918,1.34915
2018-01-02 00:03:00,1.34915,1.34940,1.20026
2018-01-02 00:03:10,NaN,NaN,NaN


In [44]:
#para cambiar los valores NaN por los que hay en la señal EUR/USD
X_GBPUSD = X_GBPUSD.fillna(X_EURUSD,axis=0)

se unen para entrenar la multiseñal...

In [45]:
X_EURUSD.insert(loc=3,column=3,value=X_GBPUSD.iloc[:,0])
X_EURUSD.insert(loc=4,column=4,value=X_GBPUSD.iloc[:,1])
X_EURUSD.insert(loc=5,column=5,value=X_GBPUSD.iloc[:,2])

In [46]:
X_EURUSD.shape

(156579, 8)

In [47]:
X_EURUSD.head()

,0,1,2,3,4,5,PNL_0,PNL_1
date,,,,,,,,
2018-01-01 23:58:20,1.20093,1.20093,1.20082,1.20093,1.20093,1.20082,-0.00028,-0.00068
2018-01-01 23:58:30,1.20093,1.20082,1.20043,1.20093,1.20082,1.20043,-0.00070,-0.00099
2018-01-01 23:58:50,1.20082,1.20043,1.20011,1.20082,1.20043,1.20011,-0.00099,-0.00098
2018-01-01 23:59:00,1.20043,1.20011,1.20015,1.20043,1.20011,1.20015,-0.00095,-0.00082
2018-01-01 23:59:10,1.20011,1.20015,1.20028,1.20011,1.20015,1.20028,-0.00082,-0.00095


se prueba con el GaussianNB

In [48]:
%run ../code/PNLEstimatorWrapper.py

In [49]:
from sklearn.naive_bayes import GaussianNB
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/GaussianNB/pnl-EUR-USD_GBP-USD_GNB_wsize3_2018-01.csv')

--------- GaussianNB ----------------
average PNL :  -0.270821052632
average buy:  1970.57894737
average sell:  5238.0
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.23772   389.0  6069.0          12.0         2887.0
2018-01-08 -0.22527  5426.0  1088.0        4748.0           22.0
2018-01-09 -0.21091  6655.0     0.0        6655.0            0.0
2018-01-10 -0.24318   161.0  6859.0           2.0         1590.0
2018-01-11 -0.24062   128.0  6984.0           2.0         3338.0
2018-01-12 -0.25766     0.0  7093.0           0.0         7093.0
2018-01-15 -0.30286  7099.0     0.0        7099.0            0.0
2018-01-16 -0.25638   983.0  6139.0           6.0           58.0
2018-01-17 -0.30150   843.0  6696.0          17.0           74.0
2018-01-18 -0.27079  1087.0  6271.0          12.0           47.0
2018-01-19 -0.29169   929.0  6382.0         160.0         3967.0
2018-01-22 -0.26221  1921.0  5230.0          40.0         1392.

se prueba con el KNeighborsClassifier

In [51]:
from sklearn.neighbors import KNeighborsClassifier
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('--------- KNeighborsClassifier -------------')
print('average PNL : ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/KNeighborsClassifier/pnl-EUR-USD_GBP-USD_KNC_wsize3_2018-01.csv')

--------- KNeighborsClassifier -------------
average PNL :  -0.270507894737
average buy:  3385.26315789
average sell:  3823.31578947
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.23020  3176.0  3282.0          60.0          100.0
2018-01-08 -0.22272  5568.0   946.0         665.0           24.0
2018-01-09 -0.21364  5658.0   997.0        1560.0           28.0
2018-01-10 -0.24078  3851.0  3169.0         202.0           58.0
2018-01-11 -0.23922  4188.0  2924.0          72.0          100.0
2018-01-12 -0.26349   852.0  6241.0          21.0          960.0
2018-01-15 -0.29640  5180.0  1919.0         132.0           54.0
2018-01-16 -0.25816  3983.0  3139.0         146.0           61.0
2018-01-17 -0.29403  4791.0  2748.0         132.0          142.0
2018-01-18 -0.27874  2737.0  4621.0          72.0           85.0
2018-01-19 -0.28409  3483.0  3828.0          58.0           60.0
2018-01-22 -0.26962  3156.0  3995.0          41.0

con el DecisionTreeClassifier...

In [52]:
from sklearn.tree import DecisionTreeClassifier
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_1','PNL_0'])
result_DTC = EU.step_validation(est_DTC, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('--------- DecisionTreeClassifier -------------')
print('average PNL : ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/DecisionTreeClassifier/pnl-EUR-USD_GBP-USD_DTC_wsize3_2018-01.csv')

--------- DecisionTreeClassifier -------------
average PNL :  -0.271414736842
average buy:  3306.52631579
average sell:  3902.05263158
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.22916  2718.0  3740.0          25.0           38.0
2018-01-08 -0.23077  1381.0  5133.0          39.0          727.0
2018-01-09 -0.21748  6038.0   617.0        4216.0           20.0
2018-01-10 -0.23790  4740.0  2280.0         236.0           25.0
2018-01-11 -0.24795  3865.0  3247.0          42.0           99.0
2018-01-12 -0.28673   788.0  6305.0          12.0          960.0
2018-01-15 -0.30060  5150.0  1949.0         132.0           92.0
2018-01-16 -0.25312  3870.0  3252.0         158.0           61.0
2018-01-17 -0.28146  4712.0  2827.0          91.0           47.0
2018-01-18 -0.28716  2592.0  4766.0          44.0           84.0
2018-01-19 -0.29458  3439.0  3872.0          25.0           75.0
2018-01-22 -0.26172  4334.0  2817.0          92

con un RandomForestClassifier...

In [53]:
from sklearn.ensemble import RandomForestClassifier
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('--------- RandomForestClassifier -------------')
print('average PNL : ', np.mean(result_RFC[:,0]))
print('average buy: ', np.mean(result_RFC[:,1]))
print('average sell: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/RandomForestClassifier/pnl-EUR-USD_GBP-USD_RFC_wsize3_2018-01.csv')

--------- RandomForestClassifier -------------
average PNL :  -0.272706315789
average buy:  3108.68421053
average sell:  4099.89473684
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.23229  3002.0  3456.0          43.0           29.0
2018-01-08 -0.23636  5063.0  1451.0          57.0           14.0
2018-01-09 -0.21125  4309.0  2346.0         481.0           26.0
2018-01-10 -0.23885  3906.0  3114.0          47.0           26.0
2018-01-11 -0.25240  4011.0  3101.0          45.0           93.0
2018-01-12 -0.28930   755.0  6338.0          17.0          960.0
2018-01-15 -0.30026  1158.0  5941.0          53.0         1880.0
2018-01-16 -0.25827  3971.0  3151.0         180.0           36.0
2018-01-17 -0.29853  4971.0  2568.0         112.0          138.0
2018-01-18 -0.28264  3549.0  3809.0          48.0           38.0
2018-01-19 -0.27896  3748.0  3563.0          39.0           41.0
2018-01-22 -0.25543  2822.0  4329.0          31